In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gzip
import shutil

from NREL_All_Stations_API import get_data

In [3]:
API_URL = "https://developer.nrel.gov/api/alt-fuel-stations/v1.csv"
API_PARAMS = {'api_key': 'EUe0n9InavfhKtKtmscW1Emd5b3IhaJwOkcHu3MN', 'fuel_type': 'ELEC'}
RAW_FILE = "./data/NREL_All_Stations_data.csv"

In [4]:
# get_data(API_URL, API_PARAMS, RAW_FILE)

In [5]:
raw_data = pd.read_csv(RAW_FILE, delimiter='\t')

/tmp/ipykernel_14132/3471019757.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv(RAW_FILE, delimiter='\t')


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35665 entries, 0 to 35664
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Station Name                 35665 non-null  object 
 1   Street Address               35665 non-null  object 
 2   Status Code                  35665 non-null  object 
 3   Expected Date                1146 non-null   object 
 4   Groups With Access Code      35665 non-null  object 
 5   Access Days Time             32474 non-null  object 
 6   Cards Accepted               4622 non-null   object 
 7   EV Level1 EVSE Num           54 non-null     float64
 8   EV Level2 EVSE Num           10772 non-null  float64
 9   EV DC Fast Count             27180 non-null  float64
 10  EV Other Info                7 non-null      object 
 11  EV Network                   35665 non-null  object 
 12  EV Network Web               34224 non-null  object 
 13  Latitude        

In [7]:
accepted_params = [
    'Station Name', 'Street Address',
    'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4',
    'Station Phone', 'Status Code', 'Expected Date',
    'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 
    'EV Other Info', 'EV Network',
    'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude',
    'Date Last Confirmed', 'Updated At', 'Owner Type Code',
    'Federal Agency ID', 'Federal Agency Name', 'Open Date',
    'EV Connector Types', 'Access Code', 'Access Detail Code',
    'Federal Agency Code', 'Facility Type', 
    'EV Pricing', 'EV On-Site Renewable Source', 'Restricted Access',
    'NPS Unit Name', 'Maximum Vehicle Class', 'EV Workplace Charging'
    ]
# raw_data = raw_data[accepted_params]
# raw_data.to_csv('./data/NREL_All_Stations_data_selected.csv', index=False, sep='\t')

In [8]:
core_params = [
    'Station Name', 'Street Address', 'State', 'Status Code', 'Expected Date',
    'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 
    'EV Other Info', 'EV Network',
    'EV Network Web', 'Latitude', 'Longitude',
    'Date Last Confirmed', 'Owner Type Code', 'Open Date',
    'EV Connector Types', 'Access Code', 'Access Detail Code', 'Facility Type', 
    'EV Pricing', 'EV On-Site Renewable Source', 'Restricted Access',
    'NPS Unit Name', 'Maximum Vehicle Class', 'EV Workplace Charging'
    ]
# raw_data = raw_data[core_params]
# raw_data.to_csv('./data/NREL_All_Stations_data_si618.csv', index=False, sep='\t')

In [9]:
test_params = [
    'Station Name', 'Street Address', 'City', 'State', 'Status Code', 'Groups With Access Code', 'Access Days Time', 'Cards Accepted',
    'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 
    'EV Network', 'EV Network Web', 'Latitude', 'Longitude',
    'Owner Type Code', 'Open Date',
    'EV Connector Types', 'Facility Type', 
    'EV Pricing', 'Restricted Access',
    'Maximum Vehicle Class', 'EV Workplace Charging'
    ]
raw_data = raw_data[test_params]
raw_data.Latitude = raw_data.Latitude.astype(float)
raw_data.Longitude = raw_data.Longitude.astype(float)

KeyError: "['City', 'State'] not in index"

In [ ]:
raw_data.describe()

,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,Latitude,Longitude
count,727.000000,59253.000000,9238.000000,67622.000000,67622.000000
mean,4.444292,2.371087,4.059104,37.749384,-96.834435
std,9.528520,3.132578,4.962888,5.066056,19.562725
min,1.000000,1.000000,1.000000,0.000000,-164.848855
25%,1.000000,2.000000,1.000000,34.027712,-118.016642
50%,2.000000,2.000000,2.000000,38.397025,-93.224228
75%,3.000000,2.000000,6.000000,41.402666,-79.047833
max,121.000000,338.000000,84.000000,64.852466,122.373276


In [ ]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67622 entries, 0 to 67621
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Station Name             67621 non-null  object 
 1   Street Address           67589 non-null  object 
 2   City                     67618 non-null  object 
 3   State                    67610 non-null  object 
 4   Status Code              67622 non-null  object 
 5   Groups With Access Code  67622 non-null  object 
 6   Access Days Time         60507 non-null  object 
 7   Cards Accepted           5357 non-null   object 
 8   EV Level1 EVSE Num       727 non-null    float64
 9   EV Level2 EVSE Num       59253 non-null  float64
 10  EV DC Fast Count         9238 non-null   float64
 11  EV Network               67619 non-null  object 
 12  EV Network Web           58176 non-null  object 
 13  Latitude                 67622 non-null  float64
 14  Longitude             

In [ ]:
raw_data['Cards Accepted'].value_counts().head(10)

Cards Accepted
A D Debit M V                         2859
A ACCOUNT_BALANCE D M V                674
A ANDROID_PAY APPLE_PAY CREDIT D M     524
A CREDIT D M V                         360
A D M V                                201
CREDIT M V                             100
Proprietor                              74
CREDIT                                  72
A CREDIT D Debit M V                    61
ACCOUNT_BALANCE CREDIT                  36
Name: count, dtype: int64

In [ ]:
raw_data.shape

(67622, 23)

In [ ]:
def plot_states():
    states_grouped = raw_data.groupby('State').count().reset_index()
    states_grouped = states_grouped[['State', 'Status Code']]
    states_sorted = states_grouped.sort_values(by='Status Code', ascending=False)
    states_sorted = states_sorted.reset_index(drop=True)
    plt.figure(figsize=(15,10))
    hist_plot = sns.countplot(x='State', data=raw_data, order=states_sorted['State'])
    hist_plot.set_xticklabels(hist_plot.get_xticklabels(), rotation=90)
    hist_plot.set_title('Number of Charging Stations by State')
    plt.show()

# plot_states()

In [ ]:
def plot_vehicles():
    hist = sns.histplot(data=raw_data, x='Maximum Vehicle Class')
    hist.set_title('Number of Charging Stations by Maximum Vehicle Class')
    hist.set_yscale('log')
    plt.show()

# plot_vehicles()

In [ ]:
def plot_facilities():
    facility_grouped = raw_data.groupby('Facility Type').count().reset_index()
    facility_grouped = facility_grouped[['Facility Type', 'Status Code']]
    facility_sorted = facility_grouped.sort_values(by='Status Code', ascending=False)
    facility_sorted = facility_sorted.reset_index(drop=True)
    facility_sorted = facility_sorted[:10]
    hist = sns.countplot(data=raw_data, x='Facility Type', order=facility_sorted['Facility Type'])
    hist.set_title('Number of Charging Stations by Top 10 Facility Type')
    hist.set_xticklabels(hist.get_xticklabels(), rotation=90)
    plt.show()

# plot_facilities()

In [ ]:
# sns.countplot(x='EV Workplace Charging', data=raw_data, orient='v')

In [ ]:
def plot_types():
    fig, ax = plt.subplots(3, 1, figsize=(10, 5))
    plt.subplots_adjust(top=2)
    ax = ax.flatten()
    bins = range(0, 150, 10)
    sns.histplot(data=raw_data, x='EV Level1 EVSE Num', bins=bins, ax=ax[0])
    ax[0].set_yscale('log')
    ax[0].set_ylim(1, 100000)
    ax[0].set_title('Number of Charging Stations by Type')
    sns.histplot(data=raw_data, x='EV Level2 EVSE Num', bins=bins, ax=ax[1])
    ax[1].set_yscale('log')
    ax[1].set_ylim(1, 100000)
    sns.histplot(data=raw_data, x='EV DC Fast Count', bins=bins, ax=ax[2])
    ax[2].set_yscale('log')
    ax[2].set_ylim(1, 100000)

# plot_types()

In [ ]:
# raw_data.to_csv('./data/NREL_All_Stations_data_si650.csv', index=False, sep='\t')

In [ ]:
google_data = pd.read_csv('./data/Google_Map_review_data.csv')
google_data.lat = google_data.lat.astype(float)
google_data.lng = google_data.lng.astype(float)

In [ ]:
mask = (abs(raw_data.Latitude - google_data.lat[0]) < 0.001) & (abs(raw_data.Longitude - google_data.lng[0]) < 0.001)

In [ ]:
selected = raw_data[mask]
selected

,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,EV Level2 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
11784,"Walmart 2508 - Boise, ID",8300 W. Overland Rd,Boise,ID,E,Public,24 hours daily,NaN,NaN,NaN,...,43.59142,-116.28359,NaN,2019-04-26,CHADEMO J1772COMBO,NaN,NaN,NaN,NaN,False


In [ ]:
raw_data = raw_data[raw_data['Status Code'] == 'E']
raw_data.reset_index(drop=True, inplace=True)
raw_data['docid'] = raw_data.index + 1
raw_data = raw_data[['docid'] + test_params]

In [ ]:
state = 'NY'
raw_state = raw_data[raw_data['State'] == state]
raw_state['Street Address'].fillna('None', inplace=True)
raw_state[raw_state['Street Address'].str.contains('Conserva')]
# aa[aa['Station Name'].str.contains('AIRPORT')]

/tmp/ipykernel_7991/475096227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_state['Street Address'].fillna('None', inplace=True)


,docid,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
18536,18537,ITHACA VISITORS ITHACA COLLEGE,Conservatory Dr,Ithaca,NY,E,Public,24 hours daily,NaN,NaN,...,42.420935,-76.500084,NaN,2020-06-12,J1772,NaN,NaN,NaN,NaN,False


In [ ]:
raw_data

,docid,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
0,1,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,E,Private,Fleet use only,NaN,NaN,...,34.248319,-118.387971,LG,1999-10-15,CHADEMO J1772 J1772COMBO,UTILITY,NaN,NaN,NaN,True
1,2,LADWP - West LA District Office,1394 S Sepulveda Blvd,Los Angeles,CA,E,Private,NaN,NaN,NaN,...,34.052542,-118.448504,LG,2020-02-28,J1772,UTILITY,Free,NaN,NaN,True
2,3,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,E,Public,5:30am-9pm; pay lot,NaN,NaN,...,34.040539,-118.271387,P,1995-08-30,J1772,PARKING_GARAGE,Free; parking fee,False,LD,False
3,4,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,E,Private,For fleet and employee use only,NaN,NaN,...,34.059133,-118.248589,LG,1999-10-15,CHADEMO J1772 J1772COMBO,UTILITY,NaN,NaN,LD,True
4,5,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,E,Private,Fleet use only,NaN,NaN,...,33.759802,-118.096665,LG,2018-05-01,CHADEMO J1772 J1772COMBO,UTILITY,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63227,63228,MONROE COUNTY HIGHLAND S1,171 Reservoir Ave Monroe County Parks Office,Rochester,NY,E,Public,24 hours daily,NaN,NaN,...,43.130883,-77.608427,NaN,2023-11-14,J1772,NaN,NaN,NaN,NaN,False
63228,63229,Riverside Buick GMC Cadillac of Cartersville,125 S DIXIE AVE,CARTERSVILLE,GA,E,Public,NaN,NaN,NaN,...,34.162794,-84.789602,NaN,2023-11-14,J1772COMBO,NaN,NaN,NaN,NaN,False
63229,63230,Seth Wadley Chevrolet Buick GMC Cadillac,400 BALLARD RD,Pauls Valley,OK,E,Public,NaN,NaN,NaN,...,34.743534,-97.265013,NaN,2023-11-14,J1772 J1772COMBO,NaN,NaN,NaN,NaN,False
63230,63231,Phil Long Ford Chapel Hill,1565 Auto Mall Loop,colorado springs,CO,E,Public,NaN,NaN,NaN,...,38.952616,-104.795859,NaN,2023-11-14,J1772COMBO,NaN,NaN,NaN,NaN,False


In [ ]:
raw_data[raw_data['docid'] == 6702]

,docid,Station Name,Street Address,City,State,Status Code,Groups With Access Code,Access Days Time,Cards Accepted,EV Level1 EVSE Num,...,Latitude,Longitude,Owner Type Code,Open Date,EV Connector Types,Facility Type,EV Pricing,Restricted Access,Maximum Vehicle Class,EV Workplace Charging
6701,6702,Meijer - Tesla Supercharger,3145 Ann Arbor-Saline Road,Ann Arbor,MI,E,Public,24 hours daily,NaN,NaN,...,42.241125,-83.766522,P,2015-06-01,TESLA,NaN,NaN,NaN,NaN,False


In [ ]:
def make_jsonl():
    raw_data.reset_index(drop=True, inplace=True)
    concatenated = raw_data.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    df_conc = pd.DataFrame(concatenated, columns=['text'])
    df_conc['docid'] = df_conc.index + 1
    df_conc = df_conc[['docid', 'text']]
    df_conc.to_json('./data/NREL_All_Stations_data_available.jsonl', lines=True, orient='records')

make_jsonl()